In [1]:
# CUDA_VISIBLE_DEVICES=0,1,2,3,4,5,6  python -m torch.distributed.launch --nproc_per_node=7 example_Imagenet_SResNet18.py


#direcyt train on imagenet

from __future__ import print_function
import argparse, pickle, torch, time, os,sys
sys.path.append("..")
import LIAF
from LIAFnet.LIAFResNet import *
from util.util import lr_scheduler

from tensorboardX import SummaryWriter
from datasets.es_imagenet import ESImagenet_Dataset

import torchvision
import torchvision.transforms as transforms
import torch.distributed as dist 
import torch.nn as nn
from torch.nn.parallel import DistributedDataParallel as DDP
from torch.cuda.amp import autocast

from importlib import import_module
import numpy as np


##################### Step2. load in dataset #####################

modules = import_module('LIAFnet.LIAFResNet_18')
config  = modules.Config()
workpath = os.path.abspath(os.getcwd())
config.cfgCnn = [3, 64, 7, True]
config.batch_size = 256//7
config.num_epochs = 50
config.actFun= LIAF.LIFactFun.apply
config.attention_model=None

batch_size = config.batch_size
num_epochs = config.num_epochs


best_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch
training_iter = 0
start_epoch = 0
acc_record = list([])
loss_train_record = list([])
loss_test_record = list([])


snn = LIAFResNet(config)
print("Total number of paramerters in networks is {}  ".format(sum(x.numel() for x in snn.parameters())))
snn=torch.nn.SyncBatchNorm.convert_sync_batchnorm(snn)


print('using uniformed init')
checkpoint = torch.load('../../saved_model/CNN_ResNet18/64.pkl', map_location=torch.device('cpu'))
#checkpoint = torch.load('./ResNet18_imagenet_LIF/16.pkl', map_location=torch.device('cpu'))
print(checkpoint)

snn.load_state_dict(checkpoint)




Total number of paramerters in networks is 11696385  
using uniformed init
OrderedDict([('conv1.decay', tensor([0.5000])), ('conv1.kernel.weight', tensor([[[[-7.8726e-03, -2.1504e-02, -5.1041e-03,  ...,  5.1610e-02,
           -3.2957e-03, -7.8043e-03],
          [-1.1625e-02,  2.7796e-02, -2.4185e-02,  ...,  4.1763e-02,
           -4.4973e-02,  6.9142e-02],
          [ 3.5329e-03,  2.2992e-03,  3.2634e-02,  ...,  2.4272e-01,
           -1.8379e-01, -1.1713e-02],
          ...,
          [ 2.6342e-03, -1.1346e-01,  2.0155e-01,  ..., -8.5931e-01,
            5.8215e-01, -3.9518e-02],
          [-1.4880e-02, -3.4547e-02, -3.3413e-01,  ..., -2.8455e-01,
           -2.8757e-01,  2.2934e-01],
          [-3.0937e-02,  1.2074e-01, -1.3569e-01,  ...,  3.2331e-01,
           -2.0518e-01,  4.4805e-02]],

         [[ 8.8028e-03, -1.2892e-02, -2.1979e-02,  ...,  7.9543e-02,
            7.5205e-03, -3.4137e-02],
          [-3.2725e-02,  5.6090e-02, -7.7457e-03,  ...,  8.3197e-02,
           -3.8855

RuntimeError: Error(s) in loading state_dict for LIAFResNet:
	Missing key(s) in state_dict: "layer1.0.cv1.decay", "layer1.0.cv1.kernel.weight", "layer1.0.cv1.kernel.bias", "layer1.0.cv1.NormLayer.weight", "layer1.0.cv1.NormLayer.bias", "layer1.0.cv1.NormLayer.running_mean", "layer1.0.cv1.NormLayer.running_var", "layer1.0.cv2.decay", "layer1.0.cv2.kernel.weight", "layer1.0.cv2.kernel.bias", "layer1.0.cv2.NormLayer.weight", "layer1.0.cv2.NormLayer.bias", "layer1.0.cv2.NormLayer.running_mean", "layer1.0.cv2.NormLayer.running_var", "layer1.1.cv1.decay", "layer1.1.cv1.kernel.weight", "layer1.1.cv1.kernel.bias", "layer1.1.cv1.NormLayer.weight", "layer1.1.cv1.NormLayer.bias", "layer1.1.cv1.NormLayer.running_mean", "layer1.1.cv1.NormLayer.running_var", "layer1.1.cv2.decay", "layer1.1.cv2.kernel.weight", "layer1.1.cv2.kernel.bias", "layer1.1.cv2.NormLayer.weight", "layer1.1.cv2.NormLayer.bias", "layer1.1.cv2.NormLayer.running_mean", "layer1.1.cv2.NormLayer.running_var", "layer2.0.cv1.decay", "l

In [2]:
print(snn.state_dict())



OrderedDict([('conv1.decay', tensor([0.5000])), ('conv1.kernel.weight', tensor([[[[-7.8726e-03, -2.1504e-02, -5.1041e-03,  ...,  5.1610e-02,
           -3.2957e-03, -7.8043e-03],
          [-1.1625e-02,  2.7796e-02, -2.4185e-02,  ...,  4.1763e-02,
           -4.4973e-02,  6.9142e-02],
          [ 3.5329e-03,  2.2992e-03,  3.2634e-02,  ...,  2.4272e-01,
           -1.8379e-01, -1.1713e-02],
          ...,
          [ 2.6342e-03, -1.1346e-01,  2.0155e-01,  ..., -8.5931e-01,
            5.8215e-01, -3.9518e-02],
          [-1.4880e-02, -3.4547e-02, -3.3413e-01,  ..., -2.8455e-01,
           -2.8757e-01,  2.2934e-01],
          [-3.0937e-02,  1.2074e-01, -1.3569e-01,  ...,  3.2331e-01,
           -2.0518e-01,  4.4805e-02]],

         [[ 8.8028e-03, -1.2892e-02, -2.1979e-02,  ...,  7.9543e-02,
            7.5205e-03, -3.4137e-02],
          [-3.2725e-02,  5.6090e-02, -7.7457e-03,  ...,  8.3197e-02,
           -3.8855e-02,  5.4726e-02],
          [-1.9549e-02, -1.0310e-02,  2.9048e-02,  ...,